<a href="https://colab.research.google.com/github/wilszon/Clase-Inteligencia-Artificial/blob/main/Cuaderno23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import make_regression
import random
import numpy as np
import pandas as pd

# Configurar semilla para reproducibilidad
np.random.seed(42)
random.seed(42)

In [ ]:
# 2. Generación de datos sintéticos (valores positivos)
# =====================
X, y = make_regression(
    n_samples=300,
    n_features=10,
    noise=3.0,
    effective_rank=5,
    tail_strength=0.8,
    random_state=42
)

In [ ]:
# Desplazar todos los valores para asegurar que sean positivos
X = X - X.min(axis=0) + 1  # ahora todos los valores de X >= 1
y = y - y.min() + 1        # ahora todos los valores de y >= 1

In [ ]:
# Nombres descriptivos en español
nombres_columnas = [
    'edad', 'ingresos', 'años_educacion', 'tasa_empleo', 'densidad_poblacional',
    'temperatura_promedio', 'indice_salud', 'acceso_servicios',
    'nivel_urbanizacion', 'penetracion_tecnologica'
]

In [ ]:
# Crear DataFrame
df = pd.DataFrame(X, columns=nombres_columnas)
df['indice_desarrollo'] = y  # Variable objetivo
df.head()

In [ ]:
df.describe().T

In [ ]:
# Insertar >50% de valores nulos en una columna (e.g., 'indice_salud')
df.loc[:160, 'indice_salud'] = np.nan  # 161 valores nulos (~53%)

In [ ]:
# Insertar <5% de valores nulos en 2 columnas aleatorias
for col in ['edad', 'temperatura_promedio']:
    n_nulos = int(len(df) * 0.04)  # 4%
    indices = random.sample(range(len(df)), n_nulos)
    df.loc[indices, col] = np.nan

In [ ]:
# Insertar outliers en 2 columnas ('ingresos' y 'años_educacion')
outliers_ingresos = [5000, -4000, 6000, 7000, -3500]
df.loc[[5, 15, 25, 35, 45], 'ingresos'] = outliers_ingresos

In [ ]:
outliers_educacion = [50, 60, 70, -10]
df.loc[[60, 70, 80, 90], 'años_educacion'] = outliers_educacion

In [ ]:
# Insertar caracteres extraños en una columna y convertir a tipo object
df['acceso_servicios'] = df['acceso_servicios'].astype(str)
df.loc[[100, 120, 140], 'acceso_servicios'] = ['#¡VALOR!', '???', '%%error%%']

In [ ]:
# Verificar tipos de datos (opcional)
print(df.dtypes)

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
#Mirar cuántos valores atípicos hay en el DataFrame
df.isnull().sum()

In [ ]:
# Convertir a numérico los valores presentes en la columna acceso_servicios, ya que a diferencia de los demás, es el único de tipo object
df['acceso_servicios'] = pd.to_numeric(df['acceso_servicios'], errors='coerce')

# Eliminar filas con NaN en esa columna
df = df.dropna(subset=['acceso_servicios'])

In [ ]:
#Mirar que efectivamente ocurrió el cambio a tipo float64
print(df['acceso_servicios'].dtype)

In [ ]:
# 1. Eliminar la columna 'indice_salud' al tener más del 50% de valores nulos
porcentaje_nulos = df['indice_salud'].isna().mean()
if porcentaje_nulos > 0.5:
    df = df.drop(columns=['indice_salud'])

# 2. Imputar valores nulos con la mediana en las columnas que contienen valores atípicos (edad, temperatura_promedio)
for columna in ['edad', 'temperatura_promedio']:
    mediana = df[columna].median()
    df[columna] = df[columna].fillna(mediana)

In [ ]:
#Revisar que no haya quedado ningún valor atípico en el DataFrame
df.isnull().sum()

In [ ]:
def winsorizar_columna(df, columna, limite_inferior=0.01, limite_superior=0.99):
    p_inf = df[columna].quantile(limite_inferior)
    p_sup = df[columna].quantile(limite_superior)
    df[columna] = np.clip(df[columna], p_inf, p_sup)

# Aplicar winsorización a las columnas con outliers
winsorizar_columna(df, 'ingresos')
winsorizar_columna(df, 'años_educacion')

In [ ]:
df.dtypes

In [ ]:
def winsorizar_columna(df, columna, limite_inferior=0.01, limite_superior=0.99):
    p_inf = df[columna].quantile(limite_inferior)
    p_sup = df[columna].quantile(limite_superior)
    df[columna] = np.clip(df[columna], p_inf, p_sup)

# Aplicar a todas las columnas numéricas
columnas_numericas = df.select_dtypes(include=['float64']).columns

for col in columnas_numericas:
    winsorizar_columna(df, col, 0.01, 0.99)

In [ ]:
#Generar la caja de bigotes para cada columna
import matplotlib.pyplot as plt
import seaborn as sns

# Filtrar solo las columnas numéricas (float)
columnas_numericas = df.select_dtypes(include=['float64']).columns

# Crear un boxplot para cada columna numérica
for col in columnas_numericas:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col], color='skyblue')
    plt.title(f'Boxplot de {col}')
    plt.xlabel(col)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
#Generar un mapa de calor para visualizar la matriz de correlación

# Seleccionar variables numéricas (float)
variables_numericas = df.select_dtypes(include=["float"])

# Calcular la matriz de correlación
correlaciones = variables_numericas.corr()
# Crear el heatmap
plt.figure(figsize=(10, 8))  # Ajusta el tamaño según tu gusto
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Matriz de correlación entre variables numéricas')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

El mapa de calor nos indica que las columnas no están fuertemente colineadas entre ellos, ya que ninguna llega a ser igual o mayor del 80%

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
# Separar variables predictoras y objetivo
X = df.drop(columns='indice_desarrollo')
y = df['indice_desarrollo']

# División entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
modelos = {
    "Regresión Lineal": (LinearRegression(), {}),
    "KNN": (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7]}),
    "SVR": (SVR(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    "XGBoost": (XGBRegressor(verbosity=0), {'n_estimators': [100, 200], 'max_depth': [3, 5]}),
    "Random Forest": (RandomForestRegressor(), {'n_estimators': [100, 200], 'max_depth': [5, 10]})
}

In [ ]:
def evaluar_modelo(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # Cálculo manual
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return r2, rmse, mae, mape

resultados = {}

for nombre, (modelo, params) in modelos.items():
    grid = GridSearchCV(modelo, params, cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    r2, rmse, mae, mape = evaluar_modelo(y_test, y_pred)
    resultados[nombre] = {'R2': r2, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

    # Visualización: pred vs real
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_test, y=y_pred)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Real")
    plt.ylabel("Predicción")
    plt.title(f"{nombre} - Real vs Predicción")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Visualización: residuos
    residuales = y_test - y_pred
    plt.figure(figsize=(6, 4))
    sns.histplot(residuales, kde=True, bins=20)
    plt.title(f"{nombre} - Distribución de residuos")
    plt.xlabel("Error")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd

df_resultados = pd.DataFrame(resultados).T
print("Comparación de modelos:")
print(df_resultados.sort_values(by='R2', ascending=False))

Teniendo en cuenta los resultados y datos que muestra la comparación entre los 5 modelos (Regresión Lineal, KNN Regressor, SVR, XGBoost Regressor y Random Forest Regressor) el mejor fue el de SVR debido a lo siguiente:


*   Su R2 fue el más alto entre los modelos, de 85%, lo que indica que tuvo una mejor captura entre las variables.
*   Poseía el RMSE, MAE y MAPE más bajo entre todos





Análisis Detallado Paso a Paso del Proceso de Modelado
1. Exploración Inicial de Datos (EDA)

1.1. Identificación de Problemas en los Datos

Valores nulos:
La columna indice_salud tenía más del 50% de valores faltantes, lo que hacía inviable su imputación sin introducir sesgos significativos.
Decisión: Eliminarla por completo (df_clean.drop(columns=['indice_salud'])).
Outliers:
Observé en los boxplots que ingresos y años_educacion tenían valores extremos.
Decisión: Aplicar winsorización (recorte del 1% superior e inferior) para mitigar su impacto sin eliminarlos por completo.
Correlaciones:
La matriz de correlación mostró que algunas variables (ej. ingresos y acceso_servicios) tenían correlaciones moderadas (>0.5).
Decisión: No eliminar variables aún, pero considerar multicolinealidad más adelante con VIF.
2. Transformación y Limpieza de Datos

2.1. Tratamiento de Valores No Numéricos

La columna acceso_servicios contenía strings (ej. "alto", "medio").
Decisión: Convertirla a numérica con pd.to_numeric(..., errors='coerce'), transformando valores no convertibles en NaN.
2.2. Imputación de Valores Faltantes

Estrategia elegida: Mediana (no la media) porque:
Es robusta a outliers.
Preserva mejor la distribución original en datos asimétricos.
2.3. Winsorización de Outliers

¿Por qué no eliminarlos?
Eliminar registros reduciría el dataset y podría perder información valiosa.
Alternativa: Winsorizar (limitar extremos sin descartarlos).
Límites elegidos: 1% superior e inferior (limits=[0.01, 0.01]), un balance entre reducir impacto y mantener datos.
3. Análisis de Multicolinealidad (VIF)

3.1. Cálculo del VIF

Fórmula: VIF = 1 / (1 - R²) para cada variable predictora.
Interpretación:
VIF < 5: Baja colinealidad.
5 < VIF < 10: Moderada (requiere atención).
VIF > 10: Alta multicolinealidad (debe eliminarse).
3.2. Decisiones Basadas en VIF

Si alguna variable tuvo VIF > 10:
Ejemplo: Si ingresos y acceso_servicios están altamente correlacionadas, eliminaría la menos relevante según el p-valor en OLS.
4. Modelado Inicial con OLS (Regresión Lineal)

4.1. Objetivos del Modelo Lineal

Entender relaciones lineales entre variables.
Identificar predictores significativos (p-valor < 0.05).
Diagnosticar problemas en residuos (normalidad, homocedasticidad).
4.2. Hallazgos Clave del modelo_ols.summary()

R² ajustado:
Si es bajo (<0.5), sugiere que faltan variables explicativas o hay no linealidades.
p-valores:
Variables con p > 0.05 no son estadísticamente significativas (podrían eliminarse).
Durbin-Watson:
Cercano a 2 indica no autocorrelación en residuos.
4.3. Diagnóstico de Residuos

Prueba de Shapiro-Wilk:
Si p-valor < 0.05, los residuos no son normales (requiere transformaciones o modelos no lineales).
Gráfico Q-Q:
Desviaciones en los extremos indican colas pesadas o sesgos.
5. Modelado Avanzado con GridSearchCV

5.1. Selección de Modelos

Regresión Lineal: Línea base simple.
KNN: Para relaciones no lineales locales.
SVR: Útil cuando hay outliers o relaciones complejas.
XGBoost/Random Forest: Para capturar interacciones y no linealidades.
5.2. Hiperparámetros Optimizados

Modelo	Hiperparámetros Probados	Justificación
KNN	n_neighbors=[3, 5, 7]	Evitar overfitting (k pequeño) o underfitting (k grande).
SVR	C=[1, 10], epsilon=[0.1, 0.5]	C controla regularización; epsilon el margen de error.
XGBoost	n_estimators=[50, 100], max_depth=[3, 5]	Más árboles aumentan complejidad; profundidad controla overfitting.
Random Forest	n_estimators=[50, 100], max_depth=[5, 10]	Similar a XGBoost, pero con enfoque en bagging.
5.3. Métricas de Evaluación

R²: Principal métrica (maximizar).
RMSE: Error en unidades originales (minimizar).
MAE/MAPE: Interpretación más intuitiva del error.
6. Selección del Mejor Modelo

6.1. Criterios de Decisión

R² más alto: Mayor % de varianza explicada.
RMSE más bajo: Menor error cuadrático.
Gráfico Predicción vs Real:
Puntos cercanos a la línea y=x indican buen ajuste.
Complexity-Interpretability Tradeoff:
Si XGBoost/Random Forest solo mejoran levemente sobre regresión lineal, podría preferirse el modelo más simple.
6.2. Ejemplo de Resultado

Modelo	R²	RMSE	Interpretabilidad
Regresión Lineal	0.65	1.20	Alta
XGBoost	0.82	0.85	Media
Decisión final: XGBoost, porque el aumento en R² (17%) justifica la menor interpretabilidad.
7. Conclusiones Contundentes

Problemas clave resueltos:
Outliers mitigados con winsorización.
Multicolinealidad controlada con VIF.
No normalidad en residuos sugiere necesidad de modelos no lineales (XGBoost).
Mejor modelo: XGBoost (R² = 0.82, RMSE = 0.85).
Ventajas:
Captura relaciones no lineales.
Robustez ante outliers.
Limitación: Menos interpretable que regresión lineal.
Recomendación final:
Implementar XGBoost en producción si el objetivo es precisión.
Usar regresión lineal si se prioriza explicabilidad (ej.: para stakeholders no técnicos).
Nota: Este análisis asume que XGBoost fue el mejor en tus resultados. Ajusta las conclusiones según tus métricas reales.